In [1]:
%gui qt

from viewer.vtkviewer import SimpleVtkViewer, get_QApplication

import numpy as np

from occmodel import Edge, EdgeIterator, Wire, Face, FaceIterator, Solid
from geotools import Plane

### [OCCT Bottle Tutorial](http://dev.opencascade.org/doc/overview/html/occt__tutorial.html)

In [2]:
height = 70.
width = 50.
thickness = 30.

pnt1 = [-width/2.,  0.,           0.]
pnt2 = [-width/2., -thickness/4., 0.]
pnt3 = [0.,        -thickness/2., 0.]
pnt4 = [width/2.,  -thickness/4., 0.]
pnt5 = [width/2.,   0.,           0.]

edge1 = Edge().createLine(start=pnt1, end=pnt2)
edge2 = Edge().createArc3P(start=pnt2, end=pnt4, pnt=pnt3)
edge3 = Edge().createLine(start=pnt4, end=pnt5)

halfProfile = Wire([edge1, edge2, edge3])

mirrorPlane = Plane(origin=[0,0,0], xaxis=[1,0,0], yaxis=[0,0,1])
mirrorProfile = halfProfile.mirror(mirrorPlane, copy=True)

allEdges = list(EdgeIterator(halfProfile)) + list(EdgeIterator(mirrorProfile))

fullProfile = Wire().createWire(allEdges)

bottomFace = Face().createFace(fullProfile)

body = Solid().extrude(bottomFace, (0, 0, 0), (0, 0, height))
body.fillet(thickness/12.)

neckHeight = height/10
neckRadius = thickness/4
neck = Solid().createCylinder([0,0,0], [0,0,neckHeight], radius=neckRadius)
neck.translate([0, 0, height])

body.fuse(neck)

zMax = -1
neckTopFace = None
for f in FaceIterator(body):
    [x, y , z] = f.centreOfMass()
    if z >= zMax:
        neckTopFace = f
        zMax = z
        
body.shell(thickness/50., [neckTopFace], tolerance=1E-3)

t_thick = neckHeight/5
t_height = neckHeight - t_thick
t_radius = neckRadius + t_thick/4
t_pitch = t_height/2
t_angle = 0

# Note the following thread geometry is not correct.  The profile
# is wrong and there is a twist added to the profile. But it's
# kind of close and good enough for this example.
threadHelix = Edge().createHelix(pitch=t_pitch, 
                                 height=t_height,
                                 radius=t_radius,
                                 angle = t_angle)

threadFace = Face().createPolygonal([[0,    0,        t_thick/2], 
                                     [t_thick,  .0,   0], 
                                     [0,    0,      -t_thick/2]])
threadFace.translate([t_radius, 0, 0])

thread = Solid().pipe(threadFace, threadHelix)
thread.translate([0, 0, height])

body.fuse(thread)

actor = body.toVtkActor()

### VTK Viewer

The following summarizes the mouse and keyboard commands for interacting with shapes rendered in the viewer.

- __Keypress j / Keypress t:__ toggle between joystick (position sensitive) and trackball (motion sensitive) styles. In joystick style, motion occurs continuously as long as a mouse button is pressed. In trackball style, motion occurs when the mouse button is pressed and the mouse pointer moves.

- __Keypress c / Keypress a:__ toggle between camera and actor modes. In camera mode, mouse events affect the camera position and focal point. In actor mode, mouse events affect the actor that is under the mouse pointer.

- __Button 1:__ rotate the camera around its focal point (if camera mode) or rotate the actor around its origin (if actor mode). The rotation is in the direction defined from the center of the renderer's viewport towards the mouse position. In joystick mode, the magnitude of the rotation is determined by the distance the mouse is from the center of the render window.

- __Button 2:__ pan the camera (if camera mode) or translate the actor (if object mode). In joystick mode, the direction of pan or translation is from the center of the viewport towards the mouse position. In trackball mode, the direction of motion is the direction the mouse moves. (Note: with 2-button mice, pan is defined as `<Shift>`Button 1.)

- __Button 3:__ zoom the camera (if camera mode) or scale the actor (if object mode). Zoom in/increase scale if the mouse position is in the top half of the viewport; zoom out/decrease scale if the mouse position is in the bottom half. In joystick mode, the amount of zoom is controlled by the distance of the mouse pointer from the horizontal centerline of the window.

- __Keypress 3:__ toggle the render window into and out of stereo mode.  By default, red-blue stereo pairs are created. Some systems support Crystal Eyes LCD stereo glasses; you have to invoke SetStereoTypeToCrystalEyes() on the rendering window object.  Note: to use stereo you also need to pass a stereo=1 keyword argument to the render window object constructor.

- __Keypress e:__ exit the application.

- __Keypress f:__ fly to the picked point

- __Keypress p:__ perform a pick operation. The render window interactor has an internal instance of vtkCellPicker that it uses to pick.

- __Keypress r:__ reset the camera view along the current view direction. Centers the actors and moves the camera so that all actors are visible.

- __Keypress s:__ modify the representation of all actors so that they are surfaces.

- __Keypress u:__ invoke the user-defined function. Typically, this keypress will bring up an interactor that you can type commands in.

- __Keypress w:__ modify the representation of all actors so that they are wireframe.

In [3]:
try:
    a = get_QApplication([])
except:
    pass

vtkWin = SimpleVtkViewer()

vtkWin.add_actor(actor)

# If the VTK window is blank/white, click on the window and hit 'r' to zoom to fit.